In [2]:
import numpy as np
import pandas as pd

import s2cell
from datetime import datetime as dt
from math import sin, cos

In [3]:
def transform(X):
    result = X.copy()
    result[['s2token', 'date', 'hour']] = X.Ids.str.split('_', expand=True)
    result[['year', 'month', 'day']] = result['date'].str.split('-', expand=True)
    result['lat'] = result['s2token'].apply(lambda x: s2cell.token_to_lat_lon(x)[0])
    result['long'] = result['s2token'].apply(lambda x: s2cell.token_to_lat_lon(x)[1])
    result['x'] = result['lat'].apply(cos) * result['long'].apply(cos)
    result['y'] = result['lat'].apply(cos) * result['long'].apply(sin)
    result['z'] = result['lat'].apply(sin)
    
    # Convert datatypes
    result['date'] = pd.to_datetime(result['date'])
    result[['hour', 'month', 'day']] = result[['hour', 'month', 'day']].astype(int)
    
    # Add
    ## weekend
    result['weekend'] = result['date'].apply(lambda x: x.weekday() >= 5)
    
    ## holiday Sep2020 - Nov2020 (No holiday in Nov2020)
#     start = dt.strptime("2020-10-28", "%Y-%m-%d")
#     end = dt.strptime("2020-10-30", "%Y-%m-%d")
#     result['holiday'] = result['date'].apply(lambda x: start <= x <= end)

    ## working_hour
    result['working_hour'] = result['hour'].apply(lambda x: 9<=x<=17)
    
    ## high_commute
    result['high_commute'] = result['hour'].apply(lambda x: 7<=x<=8 or 18<=x<=19)
    
    # Delete
    result = result.drop(['Ids', 's2token', 'year', 'lat', 'long', 'date'], axis=1)
    
    return result